In [ ]:
%load_ext autoreload

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') # NOTSET, DEBUG, INFO, WARN, ERROR, CRITICAL

from JPAS_DA import global_setup
from JPAS_DA.data import loading_tools
from JPAS_DA.data import cleaning_tools
from JPAS_DA.data import crossmatch_tools

import numpy as np
import os

from JPAS_DA.utils import plotting_utils
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
plt.close('all')
font, rcnew = plotting_utils.matplotlib_default_config()
mpl.rc('font', **font)
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')
%matplotlib widget

In [ ]:
root_path = global_setup.DATA_path
load_JPAS_x_DESI_Raul   = global_setup.load_JPAS_x_DESI_Raul
load_DESI_mocks_Raul    = global_setup.load_DESI_mocks_Raul
load_Ignasi             = global_setup.load_Ignasi

random_seed_load = global_setup.default_seed

In [ ]:
DATA = loading_tools.load_data_bundle(
    root_path=root_path,
    include=["JPAS_x_DESI_Raul", "DESI_mocks_Raul", "Ignasi"],
    JPAS_x_DESI_Raul={"datasets": load_JPAS_x_DESI_Raul},
    DESI_mocks_Raul={"datasets": load_DESI_mocks_Raul},
    Ignasi={"datasets": load_Ignasi},
    random_seed=random_seed_load,
)

In [ ]:
print(DATA["Ignasi"]['all_pd'].keys())
print(DATA["JPAS_x_DESI_Raul"]['all_pd'].keys())
print(DATA["DESI_mocks_Raul"]['all_pd'].keys())

print(DATA["Ignasi"]['all_observations'].shape)
print(DATA["JPAS_x_DESI_Raul"]['all_observations'].shape)
print(DATA["DESI_mocks_Raul"]['all_observations'].shape)

In [ ]:
config_dict_cleaning = global_setup.config_dict_cleaning

config_dict_cleaning["selection"]["i_band_sn_threshold"] = 200.0

In [ ]:
DATA = cleaning_tools.clean_data_pipeline(DATA, config=config_dict_cleaning, in_place=True)

In [ ]:
print(DATA["Ignasi"].keys())
print(DATA["JPAS_x_DESI_Raul"].keys())
print(DATA["DESI_mocks_Raul"].keys())

print(DATA["Ignasi"]['all_pd'].keys())
print(DATA["JPAS_x_DESI_Raul"]['all_pd'].keys())
print(DATA["DESI_mocks_Raul"]['all_pd'].keys())

print(DATA["Ignasi"]['all_observations'].shape)
print(DATA["JPAS_x_DESI_Raul"]['all_observations'].shape)
print(DATA["DESI_mocks_Raul"]['all_observations'].shape)

In [ ]:
Dict_LoA = {"intersection": {}, "outersection": {}}
IDs_only_DESI, IDs_only_JPAS, IDs_both, \
Dict_LoA["outersection"]["DESI_mocks_Raul"], Dict_LoA["outersection"]["JPAS_x_DESI_Raul"], \
Dict_LoA["intersection"]["DESI_mocks_Raul"], Dict_LoA["intersection"]["JPAS_x_DESI_Raul"] = crossmatch_tools.crossmatch_IDs_two_datasets(
    DATA["DESI_mocks_Raul"]['all_pd']['TARGETID'], DATA["JPAS_x_DESI_Raul"]['all_pd']['TARGETID']
)

In [ ]:
print("=== Outersection ===")
print(f"DESI_mocks_Raul groups: {len(Dict_LoA['outersection']['DESI_mocks_Raul'])}")
print(f"DESI_mocks_Raul total IDs: {len(np.concatenate(Dict_LoA['outersection']['DESI_mocks_Raul']))}")
print()
print(f"JPAS_x_DESI_Raul groups: {len(Dict_LoA['outersection']['JPAS_x_DESI_Raul'])}")
print(f"JPAS_x_DESI_Raul total IDs: {len(np.concatenate(Dict_LoA['outersection']['JPAS_x_DESI_Raul']))}")

print("\n=== Intersection ===")
print(f"DESI_mocks_Raul groups: {len(Dict_LoA['intersection']['DESI_mocks_Raul'])}")
print(f"DESI_mocks_Raul total IDs: {len(np.concatenate(Dict_LoA['intersection']['DESI_mocks_Raul']))}")
print()
print(f"JPAS_x_DESI_Raul groups: {len(Dict_LoA['intersection']['JPAS_x_DESI_Raul'])}")
print(f"JPAS_x_DESI_Raul total IDs: {len(np.concatenate(Dict_LoA['intersection']['JPAS_x_DESI_Raul']))}")

In [ ]:
# --- Settings ---
survey_jpas = "JPAS_x_DESI_Raul"
survey_desi = "DESI_mocks_Raul"
NN = 1  # samples per spectral class

# --- Pull arrays we need
ids_jpas = np.asarray(DATA[survey_jpas]['all_pd']['TARGETID'])
ids_desi = np.asarray(DATA[survey_desi]['all_pd']['TARGETID'])

# --- Crossmatch to get intersection and per-ID indices
IDs_only_1, IDs_only_2, IDs_both, idxs_only_1, idxs_only_2, idxs_both_jpas, idxs_both_desi = (
    crossmatch_tools.crossmatch_IDs_two_datasets(ids_jpas, ids_desi)
)

# --- Build a quick lookup from TARGETID -> (indices in JPAS, indices in DESI)
id2idx = {}
for tid, ij, idesi in zip(IDs_both, idxs_both_jpas, idxs_both_desi):
    id2idx[tid] = {survey_jpas: np.atleast_1d(ij), survey_desi: np.atleast_1d(idesi)}

# --- Build the "matched JPAS" flat index list to choose samples by class
matched_JPAS_idx = np.concatenate(idxs_both_jpas) if len(idxs_both_jpas) else np.array([], dtype=int)

matched_SPECTYPE_int = np.asarray(DATA[survey_jpas]['all_pd']['SPECTYPE_int'])[matched_JPAS_idx]
matched_SPECTYPE_str = np.asarray(DATA[survey_jpas]['all_pd']['SPECTYPE'])[matched_JPAS_idx]
matched_TARGETID     = ids_jpas[matched_JPAS_idx]

# --- Unique classes (by integer code), and a stable string name per code
uniq_ints = np.unique(matched_SPECTYPE_int)
# map int code -> a representative string label (first seen)
int2label = {}
for u in uniq_ints:
    pos = np.flatnonzero(matched_SPECTYPE_int == u)
    int2label[int(u)] = str(matched_SPECTYPE_str[pos[0]]) if pos.size else f"class_{u}"

# --- Sample NN TARGETIDs per class (from the matched-JPAS subset)
rng = np.random.default_rng(0)
sampled_TARGETIDs = []
for code in uniq_ints:
    pos = np.flatnonzero(matched_SPECTYPE_int == code)
    if pos.size == 0:
        continue
    pick = rng.choice(pos, size=min(NN, pos.size), replace=False)
    sampled_TARGETIDs.append(matched_TARGETID[pick])
# shape ~ (n_classes, up to NN)
sampled_TARGETIDs = np.array(sampled_TARGETIDs, dtype=object)

# --- Plot setup (main + ratio)
fig, (ax, ax_ratio) = plt.subplots(
    2, 1, figsize=(9, 8), height_ratios=[3, 1], sharex=True, gridspec_kw={'hspace': 0.05}
)
ax.set_ylabel(r'Flux [arb. units]', fontsize=20)
ax_ratio.set_xlabel(r'$\mathrm{Filter~Index}$', fontsize=20)
ax_ratio.set_ylabel(r'$\frac{\log_{10}\mathrm{DESI}}{\log_{10}\mathrm{JPAS}}$', fontsize=18)

# Colors: one per spectral class
colors = plt.cm.plasma(np.linspace(0.05, 0.95, len(uniq_ints)))
linestyles = {survey_jpas: "--", survey_desi: "-"}

# --- Plot
label_lines_done = {survey_jpas: False, survey_desi: False}
ll = 0  # for staggered redshift text placement

for j, code in enumerate(uniq_ints):
    if j >= len(sampled_TARGETIDs):
        continue
    for k in range(len(sampled_TARGETIDs[j])):
        tid = sampled_TARGETIDs[j][k]
        if tid is None:
            continue
        # indices for this TARGETID in each dataset (use all occurrences)
        for key_survey in (survey_jpas, survey_desi):
            idxs = id2idx.get(tid, {}).get(key_survey, np.array([], dtype=int))
            for idx in np.atleast_1d(idxs):
                obs = np.asarray(DATA[key_survey]['all_observations'][idx], dtype=float)
                err = np.asarray(DATA[key_survey]['all_errors'][idx], dtype=float)
                z   = DATA[key_survey]['all_pd'].get('REDSHIFT', [np.nan])[idx]

                lbl = None
                if not label_lines_done[key_survey]:
                    lbl = "JPAS" if key_survey == survey_jpas else "DESI"
                    label_lines_done[key_survey] = True

                ax.errorbar(
                    np.arange(obs.size), obs, yerr=err,
                    capsize=2, lw=2.0, ls=linestyles[key_survey],
                    color=colors[j], ecolor=colors[j], alpha=0.85, label=lbl
                )

                # annotate redshift on JPAS curves
                if key_survey == survey_jpas and np.isfinite(z):
                    mid_idx = (ll % max(1, obs.size // 3)) * 3
                    mid_idx = min(mid_idx, obs.size - 1)
                    mid_y = obs[mid_idx] if np.isfinite(obs[mid_idx]) and obs[mid_idx] > 0 else np.nan
                    if np.isfinite(mid_y) and mid_y > 0:
                        ax.text(
                            mid_idx, mid_y * 1.1, f"z={z:.3f}",
                            fontsize=10, color=colors[j], ha='center', va='bottom',
                            bbox=dict(facecolor='white', edgecolor='none', alpha=0.65, boxstyle="round,pad=0.2")
                        )
                        ll += 1

        # ratio plot (use first occurrence in each)
        idxs_j = id2idx.get(tid, {}).get(survey_jpas, np.array([], dtype=int))
        idxs_d = id2idx.get(tid, {}).get(survey_desi, np.array([], dtype=int))
        if idxs_j.size and idxs_d.size:
            obs_j = np.asarray(DATA[survey_jpas]['all_observations'][idxs_j[0]], dtype=float)
            obs_d = np.asarray(DATA[survey_desi]['all_observations'][idxs_d[0]], dtype=float)
            # safe logs
            j_safe = np.log10(np.clip(obs_j, 1e-12, None))
            d_safe = np.log10(np.clip(obs_d, 1e-12, None))
            ratio = d_safe / np.where(j_safe == 0, np.nan, j_safe)
            ax_ratio.plot(np.arange(ratio.size), ratio, color=colors[j], ls="-", lw=1.5, alpha=0.85)

# Styling
ax.axhline(0, ls='--', lw=0.7, color='black', alpha=0.4)
ax.set_yscale("log")
ax.tick_params(axis='both', labelsize=13)
ax_ratio.axhline(1.0, ls='--', lw=1.0, color='black', alpha=0.6)
ax_ratio.tick_params(axis='both', labelsize=12)

# Legend for survey (linestyle)
survey_handles = [
    mpl.lines.Line2D([0], [0], color="gray", linestyle=linestyles[survey_jpas], lw=2, label="JPAS"),
    mpl.lines.Line2D([0], [0], color="gray", linestyle=linestyles[survey_desi], lw=2, label="DESI"),
]
leg0 = ax.legend(handles=survey_handles, loc='upper left', fancybox=True, shadow=True,
                 fontsize=12, title="Survey", title_fontsize=13)
ax.add_artist(leg0)

# Legend for spectral classes (colors)
spectype_handles = [
    mpl.lines.Line2D([0], [0], color=colors[j], lw=4, label=int2label[int(code)])
    for j, code in enumerate(uniq_ints)
]
leg1 = ax.legend(handles=spectype_handles, loc='upper right', fancybox=True, shadow=True,
                 fontsize=12, title="Spectral Type", title_fontsize=13)
ax.add_artist(leg1)

plt.tight_layout()
plt.show()

In [ ]:
# --- Settings ---
survey_jpas = "JPAS_x_DESI_Raul"
survey_desi = "DESI_mocks_Raul"
NN = 3  # how many spectra per class to display

# Boxed class-label placement (axes fraction coords) and style
CLASS_BOX_XY = (0.78, 0.15)  # (x, y) in axes coords; top-left by default
CLASS_BOX_KW = dict(facecolor='white', alpha=0.95, edgecolor='k', boxstyle='round,pad=0.25')

# TargetID annotation style
TID_FONTSIZE = 7
TID_BOX_KW = dict(facecolor='white', alpha=0.2, edgecolor='none', boxstyle='round,pad=0.2')

# Fonts
FS_LABEL = 16
FS_TICKS = 14
FS_LEGEND = 15
FS_CLASS  = 15
FS_TITLE  = 16  # not used for per-axes .set_title (we use boxed labels instead)

# --- Pull arrays we need
ids_jpas = np.asarray(DATA[survey_jpas]['all_pd']['TARGETID'])
ids_desi = np.asarray(DATA[survey_desi]['all_pd']['TARGETID'])

# --- Crossmatch to get intersection and per-ID indices
IDs_only_1, IDs_only_2, IDs_both, idxs_only_1, idxs_only_2, idxs_both_jpas, idxs_both_desi = (
    crossmatch_tools.crossmatch_IDs_two_datasets(ids_jpas, ids_desi)
)

# --- Build a quick lookup from TARGETID -> (indices in JPAS, indices in DESI)
id2idx = {}
for tid, ij, idesi in zip(IDs_both, idxs_both_jpas, idxs_both_desi):
    id2idx[tid] = {survey_jpas: np.atleast_1d(ij), survey_desi: np.atleast_1d(idesi)}

# --- Build the "matched JPAS" flat index list to choose samples by class
matched_JPAS_idx = np.concatenate(idxs_both_jpas) if len(idxs_both_jpas) else np.array([], dtype=int)

matched_SPECTYPE_int = np.asarray(DATA[survey_jpas]['all_pd']['SPECTYPE_int'])[matched_JPAS_idx]
matched_SPECTYPE_str = np.asarray(DATA[survey_jpas]['all_pd']['SPECTYPE'])[matched_JPAS_idx]
matched_TARGETID     = ids_jpas[matched_JPAS_idx]

# --- Unique classes (by integer code), and a stable string name per code
uniq_ints = np.unique(matched_SPECTYPE_int)
int2label = {}
for u in uniq_ints:
    pos = np.flatnonzero(matched_SPECTYPE_int == u)
    int2label[int(u)] = str(matched_SPECTYPE_str[pos[0]]) if pos.size else f"class_{u}"

# --- Helper: sample UNIQUE TARGETIDs per class (up to NN)
rng = np.random.default_rng(1)
sampled_TARGETIDs = []
for code in uniq_ints:
    pos = np.flatnonzero(matched_SPECTYPE_int == code)
    if pos.size == 0:
        sampled_TARGETIDs.append(np.array([], dtype=object))
        continue

    tids_class = matched_TARGETID[pos]
    # de-duplicate TARGETIDs while preserving first-seen order
    _, first_idx = np.unique(tids_class, return_index=True)
    tids_unique = tids_class[np.sort(first_idx)]

    if tids_unique.size == 0:
        sampled_TARGETIDs.append(np.array([], dtype=object))
        continue

    pick_idx = rng.choice(tids_unique.size, size=min(NN, tids_unique.size), replace=False)
    sampled_TARGETIDs.append(tids_unique[pick_idx])
sampled_TARGETIDs = np.array(sampled_TARGETIDs, dtype=object)

# --- Plot setup: 1 row per class, shared x **and y** axes
n_classes = len(uniq_ints)
fig_h = max(3, 2.9 * n_classes)
fig, axes = plt.subplots(
    n_classes, 1, figsize=(7, fig_h), sharex=True, sharey=True, gridspec_kw={'hspace': 0.08}
)
if n_classes == 1:
    axes = np.array([axes])

# Colors: one per spectral class
# colors = plt.cm.tab10(np.linspace(0.05, 0.95, n_classes))
colors = np.array(plt.cm.get_cmap("tab10").colors[:n_classes])
linestyles = {survey_jpas: "-", survey_desi: "--"}

# Label each survey only once across all subplots
label_done = {survey_jpas: False, survey_desi: False}

# Track global y-range across ALL panels for common limits (log-scale safe)
tiny = 1e-12
global_min = np.inf
global_max = -np.inf

for j, (ax, code) in enumerate(zip(axes, uniq_ints)):
    tids_for_class = sampled_TARGETIDs[j] if j < len(sampled_TARGETIDs) else []
    ax.set_ylabel("Flux", fontsize=FS_LABEL)

    # Class label inside the panel (boxed text)
    label_text = f"{int2label[int(code)]}"
    ax.text(
        CLASS_BOX_XY[0], CLASS_BOX_XY[1], label_text,
        transform=ax.transAxes, ha='left', va='top',
        fontsize=FS_CLASS, color=colors[j], bbox=CLASS_BOX_KW
    )

    # counter to stagger TARGETID annotations across x
    ann_count = 0

    for tid in np.atleast_1d(tids_for_class):
        if tid is None:
            continue

        idxs_j = np.atleast_1d(id2idx.get(tid, {}).get(survey_jpas, np.array([], dtype=int)))
        idxs_d = np.atleast_1d(id2idx.get(tid, {}).get(survey_desi, np.array([], dtype=int)))
        if idxs_j.size == 0:
            continue  # nothing to plot

        # First occurrence only
        idx_j = idxs_j[0]
        obs_j = np.asarray(DATA[survey_jpas]['all_observations'][idx_j], dtype=float)
        err_j = np.asarray(DATA[survey_jpas]['all_errors'][idx_j], dtype=float)

        obs_d = err_d = None
        if idxs_d.size > 0:
            idx_d = idxs_d[0]
            obs_d = np.asarray(DATA[survey_desi]['all_observations'][idx_d], dtype=float)
            err_d = np.asarray(DATA[survey_desi]['all_errors'][idx_d], dtype=float)

        # DESI→JPAS scaling in flux space via mean(JPAS/DESI) over valid positives
        desi_scale = 1.0
        if obs_d is not None and obs_d.size == obs_j.size:
            valid = np.isfinite(obs_j) & np.isfinite(obs_d) & (obs_j > 0) & (obs_d > 0)
            if np.any(valid):
                desi_scale = float(np.mean(obs_j[valid] / np.clip(obs_d[valid], tiny, None)))

        # plot JPAS (flux)
        lbl_j = None if label_done[survey_jpas] else "JPAS"
        ax.errorbar(
            np.arange(obs_j.size), obs_j, yerr=err_j,
            capsize=2, lw=1.2, ls=linestyles[survey_jpas],
            color=colors[j], ecolor=colors[j], alpha=0.95, label=lbl_j
        )
        if lbl_j:
            label_done[survey_jpas] = True

        # plot DESI (flux, scaled)
        if obs_d is not None:
            obs_d_scaled = obs_d * desi_scale
            err_d_scaled = err_d * desi_scale if err_d is not None else None
            lbl_d = None if label_done[survey_desi] else "DESI (scaled)"
            ax.errorbar(
                np.arange(obs_d_scaled.size), obs_d_scaled, yerr=err_d_scaled,
                capsize=2, lw=1.2, ls=linestyles[survey_desi],
                color=colors[j], ecolor=colors[j], alpha=0.95, label=lbl_d
            )
            if lbl_d:
                label_done[survey_desi] = True

            # update global range with DESI too
            valid_d = np.isfinite(obs_d_scaled) & (obs_d_scaled > 0)
            if np.any(valid_d):
                global_min = min(global_min, float(np.min(obs_d_scaled[valid_d])))
                global_max = max(global_max, float(np.max(obs_d_scaled[valid_d])))

        # update global range with JPAS
        valid_j = np.isfinite(obs_j) & (obs_j > 0)
        if np.any(valid_j):
            global_min = min(global_min, float(np.min(obs_j[valid_j])))
            global_max = max(global_max, float(np.max(obs_j[valid_j])))

        # --- Annotate TARGETID near the JPAS curve, staggered across x
        # choose an x-index spaced across the wavelength/filter grid
        N = obs_j.size
        chunk = max(1, N // 4)  # roughly split into 4 segments
        mid_idx = min(ann_count * chunk + chunk // 2, N - 1)
        # find a nearby valid positive y for placement
        mid_idx_search = mid_idx
        for delta in range(0, min(10, N)):
            cand = mid_idx + (-1)**delta * (delta // 2)
            if 0 <= cand < N and np.isfinite(obs_j[cand]) and obs_j[cand] > 0:
                mid_idx_search = cand
                break
        y_val = obs_j[mid_idx_search]
        # place a bit above the point (works for log scale by multiplying)
        y_text = y_val * 1.15 if y_val > 0 else np.nan
        # if np.isfinite(y_text) and y_text > 0:
        #     ax.text(
        #         mid_idx_search, y_text, f"TID={int(tid)}",
        #         fontsize=TID_FONTSIZE, color=colors[j], ha='center', va='bottom',
        #         bbox=TID_BOX_KW
        #     )
        #     ann_count += 1

    ax.set_yscale("log")
    ax.tick_params(axis='both', labelsize=FS_TICKS)

# Apply shared x-label
axes[-1].set_xlabel("JPAS Narrow-Band Filter (index)", fontsize=FS_LABEL)

# Set common y-limits across all subplots (respect log scale)
if np.isfinite(global_min) and np.isfinite(global_max) and global_min > 0 and global_max > 0:
    y_lo = max(tiny, global_min / 1.4)
    y_hi = global_max * 1.4
    for ax in axes:
        ax.set_ylim(y_lo, y_hi)

# Survey legend (linestyles) – put it on the top panel
survey_handles = [
    mpl.lines.Line2D([0], [0], color="gray", linestyle=linestyles[survey_jpas], lw=3, label="JPAS Obs."),
    mpl.lines.Line2D([0], [0], color="gray", linestyle=linestyles[survey_desi], lw=3, label="Mocks"),
]
leg0 = axes[0].legend(handles=survey_handles, loc='lower left', fancybox=True, shadow=True,
                      fontsize=FS_LEGEND, title=None, title_fontsize=FS_LEGEND+1)
axes[0].add_artist(leg0)

plt.tight_layout()
plt.savefig(os.path.join(global_setup.path_saved_figures, "data_spectra.pdf"), format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
# --- helpers ---
def _to_mag(arr):
    arr = np.asarray(arr)
    mags = -2.5 * np.log10(arr) + 22.5
    return mags[np.isfinite(mags)]

def _representative_indices_from_groups(list_of_arrays, exact_unique_only=False):
    """
    Turn List-of-Arrays (indices grouped by TARGETID) into a single
    vector of representative indices: one index per TARGETID group.
    If exact_unique_only=True, keep only groups of size 1 (drop true duplicates).
    """
    reps = []
    for grp in list_of_arrays:
        if grp is None:
            continue
        grp = np.asarray(grp, dtype=int)
        if grp.size == 0:
            continue
        if exact_unique_only and grp.size != 1:
            continue
        reps.append(int(grp[0]))   # pick first as representative
    if not reps:
        return np.array([], dtype=int)
    return np.unique(np.asarray(reps, dtype=int))

# --- choose whether to keep one per TARGETID (default) or only exactly-unique TARGETIDs ---
exact_unique_only = False  # set True to drop any TARGETID that appears more than once

# --- build deduped index map per dataset ---
unique_idx_map = {}
for ds in ("DESI_mocks_Raul", "JPAS_x_DESI_Raul"):
    reps_outer = _representative_indices_from_groups(Dict_LoA["outersection"][ds], exact_unique_only=exact_unique_only)
    reps_inter = _representative_indices_from_groups(Dict_LoA["intersection"][ds], exact_unique_only=exact_unique_only)
    unique_idx = np.unique(np.concatenate([reps_outer, reps_inter]))
    unique_idx_map[ds] = unique_idx
    print(f"{ds}: using {unique_idx.size:,} unique TARGETIDs "
          f"(outer reps {reps_outer.size:,}, inter reps {reps_inter.size:,})")

# --- style maps (as you had) ---
survey_styles = {
    "JPAS_x_DESI_Raul": "-",
    "DESI_mocks_Raul":  "--",
}
survey_labels = {
    "JPAS_x_DESI_Raul": "JPAS Obs.",
    "DESI_mocks_Raul":  "Mocks",
}
labels = ["QSO_high", "QSO_low", "GALAXY", "STAR"]
class_colors = {"All": "black"} | {k: v.tolist() for k, v in zip(labels, colors)}

class_labels = {k: k for k in class_colors.keys()}

# --- build the 'series' with deduped indices ---
series = []
for ds in ("DESI_mocks_Raul", "JPAS_x_DESI_Raul"):
    idx = unique_idx_map[ds]

    # 'All'
    vals_all = _to_mag(np.asarray(DATA[ds]["all_pd"]["DESI_FLUX_R"])[idx])
    series.append({
        "values": vals_all, "survey": ds, "cls": "All",
        "label": f"{survey_labels[ds]} · All"
    })

    # per class (SPECTYPE already has QSO_high/QSO_low/GALAXY/STAR)
    spectype = np.asarray(DATA[ds]["all_pd"]["SPECTYPE"])[idx].astype(object)
    for sp in ("QSO_high", "QSO_low", "GALAXY", "STAR"):
        mask = (spectype == sp)
        if np.any(mask):
            vals = _to_mag(np.asarray(DATA[ds]["all_pd"]["DESI_FLUX_R"])[idx][mask])
            if vals.size > 0:
                series.append({
                    "values": vals, "survey": ds, "cls": sp,
                    "label": f"{survey_labels[ds]} · {sp}"
                })

# --- plot ---
fig, ax = plotting_utils.plot_multi_histograms_two_legends_new(
    series=series,
    # no ranges / range_colors → no shaded bands
    bins=30,
    survey_styles=survey_styles,
    survey_labels=survey_labels,
    class_colors=class_colors,
    class_labels=class_labels,
    x_label="DESI Magnitude (R)",
    y_label="Normalized Frequency",
    title=None,
    # x_range=None means use the full data range; here we set explicit limits below
    x_range=None,
    figsize=(9, 7),
    # legends
    show_survey_legend=True, survey_legend_loc="lower right",
    show_class_legend=True,  class_legend_loc="lower left",
    legend_fontsize=18,
    # scaling & labels
    logy=True,
    label_on_curve=True,          # one label per curve
    curve_label_mode="count",     # show only N={N}
    total_label_fmt="N={N:,}",    # e.g., N=12,345
    # disable old band boxes
    show_band_counts=False,
    # aesthetics
    linewidth=4.0,
)

# Magnitude range and y-limits as you specified
ax.set_xlim(16, 24)
ax.set_ylim(1e-5, 0.4)

fig.savefig(os.path.join(global_setup.path_saved_figures, "histogram_Mocks_Obs_class_distributions.pdf"))
plt.show()